In [ ]:
from selenium import webdriver # type: ignore
from selenium.webdriver.chrome.service import Service # type: ignore
from selenium.webdriver.common.by import By # type: ignore
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager # type: ignore
import time
url = "https://phongtro123.com/tinh-thanh/ha-noi"

name = []
thumb = []
cost = []
link_list = []
area_list = []
location_list = []
descriptions = []
features_green_list = []
image_room = []
# đảm bảo mỗi item vào view (kích hoạt lazy)

def getinfo(url):
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get(url)
    wait = WebDriverWait(driver, 10)
    items = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "ul.post__listing li")))
    for item in items:
        try:
            driver.execute_script("arguments[0].scrollIntoView({block:'center'});", item)
            time.sleep(0.15)
        except Exception:
            pass

        try:
            # title + link
            title_el = item.find_element(By.CSS_SELECTOR, "h3 a")
            title = title_el.text.strip()
            link = title_el.get_attribute("href") or ""
            name.append(title)
            link_list.append(link)
        except Exception:
            name.append("")
            link_list.append("")

        try:
            # price và diện tích (các <span> trong div.mb-2.line-clamp-1)
            spans = item.find_elements(By.CSS_SELECTOR, "div.mb-2.line-clamp-1 span")
            price = spans[0].text.strip() if spans else ""
            area = spans[-1].text.strip() if len(spans) > 1 else ""
            cost.append(price)
            area_list.append(area)
        except Exception:
            cost.append("")
            area_list.append("")

        try:
            # location
            loc = item.find_element(By.CSS_SELECTOR, "div.mb-2.d-flex a").text.strip()
            location_list.append(loc)
        except Exception:
            location_list.append("")

        # image (lazy)
        try:
            img = item.find_element(By.CSS_SELECTOR, "img")
            src = img.get_attribute("src") or img.get_attribute("data-src") or img.get_attribute("data-lazy") or ""
            thumb.append(src)
        except Exception:
            thumb.append("")
    def get_des(url):
        try:
            driver.get(url)
            wait = WebDriverWait(driver, 10)
            info = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.pb-3")))
            descriptions.append(info.text.strip())
        except Exception:
            descriptions.append("")

        # chỉ lấy các feature có tick màu xanh (icon có class 'green')
        features = []
        try:
            icons = driver.find_elements(By.CSS_SELECTOR, "i.icon.check-circle-fill.green, i.check-circle-fill.green")
            for ic in icons:
                try:
                    parent = ic.find_element(By.XPATH, "./..")
                    txt = parent.text.strip()
                    if txt:
                        features.append(txt)
                except Exception:
                    continue
        except Exception:
            pass

        features_green_list.append(features)

        imgs = []
        try:
            selectors = ".carousel-inner"
            slider = driver.find_element(By.CSS_SELECTOR, selectors)
            im = slider.find_elements(By.CSS_SELECTOR, "img")
            for i in im:
                try:
                    src = i.get_attribute("src") or i.get_attribute("data-src") or i.get_attribute("data-lazy") or im.get_attribute("data-original") or ""
                    if src and not src.strip().startswith("data:"):
                        imgs.append(src.strip())
                except Exception:
                    continue
        except Exception:
            pass
        seen = set()
        unique_imgs = []
        for u in imgs:
            if u not in seen:
                seen.add(u)
                unique_imgs.append(u)
        image_room.append(unique_imgs)

    for link in link_list:
        if not link:
            descriptions.append("")
            features_green_list.append([])
            continue
        final_url = ("https://phongtro123.com" + link) if link.startswith("/") else link
        get_des(final_url)
    driver.quit()


In [45]:
getinfo(url)
import pandas as pd
fullinfo = pd.DataFrame({
    "name": name,
    "thumb": thumb,
    "cost": cost,
    "area": area_list,
    "location": location_list,
    "link": link_list,
    "description": descriptions,
    "features_green": features_green_list,
    "images": image_room
})

In [48]:
fullinfo.to_csv("test.csv",index=False)